## Common Modulus Attack 

what will happen when the same message is encrypted twice with the same modulus but using a different public key

In [1]:
import secrets

In [2]:
def gcd(a,b):
    while(b):
        a, b = b, a % b
    return a

In [3]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = secrets.randbelow(p-2) + 2 

        while gcd(a,p) != 1:
            a = secrets.randbelow(p-2) + 2
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [4]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [5]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

In [6]:
def RSA_generate_d(e, p, q):
    N = p * q
    phiN = (p-1)*(q-1)
    assert gcd(e, phiN) == 1, "GCD of e and Phi N must be 1"
    d = pow(e, -1, phiN)

    return d


In [7]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

In [8]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


In [19]:
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x

def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError('Modular inverse does not exist.')
    else:
        return x % m

## Known to attacker<br>
e1 = public key of first cipher text<br>
c1 = first cipher text<br>
e2 = public key of the second cipher text<br>
c2 = second cipher text<br>
n = modulus that is common to both cipher text

In [9]:
p = generate_prime(512, 3)
q = generate_prime(512, 3)

N = p * q

e1 = 17 
e2 = 257 


In [10]:
d1 = RSA_generate_d(e1, p, q)
d2 = RSA_generate_d(e2, p, q)

message = "Your Auntie very pretty"
ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

89111117114032065117110116105101032118101114121032112114101116116121


226

In [11]:
encrypted_message1 = RSA_encrypt(ascii_message, e1, N)
encrypted_message2 = RSA_encrypt(ascii_message, e2, N)

### Conditions of the attack

Must make sure that these inverses exist in the first place:
- gcd(e1, e2) = 1
- gcd(c2, n) = 1

In [13]:
while True: 
    if gcd(e1, e2) != 1: 
        input('Pick new e1: ', e1)
        input('Pick new e2: ', e2)
        encrypted_message1 = RSA_encrypt(ascii_message, e1, N)
        encrypted_message2 = RSA_encrypt(ascii_message, e2, N)
    if gcd(encrypted_message2, N) != 1:
        p = generate_prime(512, 3)
        q = generate_prime(512, 3)
        N = p * q
        encrypted_message1 = RSA_encrypt(ascii_message, e1, N)
        encrypted_message2 = RSA_encrypt(ascii_message, e2, N)
    else:
        break

## Attack

Since we know e1 and e2 are coprime<br>
<br>
xe1 + ye2 = gcd(e1, e2) = 1

To solve for x and y we use extended euclidean algorithm to solve gcd(e1, e2)

Decrypt first message using x: (m^e1)^x


In [27]:
result = extended_gcd(e1, e2)
print('gcd: ', result[0])
print('coefficient x and y: ', result[1], result[2])
x = result[1]
y = result[2]

# Decrypt first encrypted message with x
decrypt_1 = pow(encrypted_message1, x, N)
print('Decrypt 1: ', decrypt_1)

# Decrypt second encrypted message with y
encrypt_inv = pow(encrypted_message2, -1, N)
decrypt_2 = pow(encrypt_inv, -y, N)
print('Decrypt 2: ', decrypt_2)

decrypt_attempt = (decrypt_1 * decrypt_2) % N

print('Hacked attempt: ', convert_to_string(decrypt_attempt))
print('Initial Message: ', message)



gcd:  1
coefficient x and y:  121 -8
Decrypt 1:  5971232700199762415948234826355095088851862549329655593785472838759376285846072002255647681701574436590564951175693440920495805093213199394682712246400646054122628037957900343337245372360879043036513954669506795963806012638591189270772295378896928609851987539099660868358768753267141232044478484320006833343
Decrypt 2:  2751143231251489219319837335963372080359138778874381119026204828365910696930591152335906620827715391438238003643503940331932546737996535685598693340691936690383490633422706358291683392061567893018667358177192589545624781028959531983732669446950353875928457823469034100679959088037469564342167031665016924712
Hacked attempt:  Your Auntie very pretty
Initial Message:  Your Auntie very pretty
